In [1]:
import numpy as np
import pandas as pd
from simple.ohlc import renko, midPrice
from simple.plotly import chartFigure
from simple.funcs import symlog

In [2]:
T = np.load('data/ETHUSDT.2022-05-08.tick.npz')['ETHUSDT'].view(np.recarray)
len(T)

2224

In [3]:
Buy = T[T.VolumeA > 0]
Sell = T[T.VolumeA < 0]
Buy[:5]

rec.array([('2022-05-08T00:01:00.335000', 0, 2636.,  6.),
           ('2022-05-08T00:01:00.335000', 1, 2636.,  4.),
           ('2022-05-08T00:01:00.668000', 2, 2636., 96.),
           ('2022-05-08T00:01:00.668000', 3, 2636.,  2.),
           ('2022-05-08T00:01:00.668000', 4, 2636.,  2.)],
          dtype={'names':['DateTimeA','LocalTimeA','PriceA','VolumeA'], 'formats':['<M8[us]','<i8','<f8','<f8'], 'offsets':[0,8,16,24], 'itemsize':40})

In [4]:
S = np.load('data/ETHUSDT.2022-05-08.bidask.npz')['ETHUSDT'].view(np.recarray)
S[:5]

rec.array([('2022-05-08T00:01:00.007000', 2636., 2635.99),
           ('2022-05-08T00:01:00.043000', 2636., 2635.99),
           ('2022-05-08T00:01:00.079000', 2636., 2635.99),
           ('2022-05-08T00:01:00.117000', 2636., 2635.99),
           ('2022-05-08T00:01:00.149000', 2636., 2635.99)],
          dtype={'names':['DT','A','B'], 'formats':['<M8[us]','<f4','<f4'], 'offsets':[0,8,16], 'itemsize':24})

In [5]:
len(T), len(S)

(2224, 1701)

In [6]:
# indexed chart
fig = chartFigure(
    Tick=dict(y=T.PriceA, x=T.LocalTimeA, color='gray', opacity=0.4),
    Sell=dict(y=Sell.PriceA, x=Sell.LocalTimeA, color='red', mode='markers', size=-symlog(Sell.VolumeA)+3),
    Buy=dict(y=Buy.PriceA, x=Buy.LocalTimeA, color='green', mode='markers', size=symlog(Buy.VolumeA)+3)
)
fig

FigureWidgetResampler({
    'data': [{'line': {'color': 'gray'},
              'name': '<b style="color:sandyb…

In [7]:
C = renko(T, 0.1)
C

[(0, 15, 2635.9, 2636.0),
 (16, 16, 2636.0, 2636.1),
 (17, 23, 2636.1000000000004, 2636.2000000000003),
 (24, 36, 2636.3, 2636.2000000000003),
 (37, 37, 2636.2000000000003, 2636.1000000000004),
 (38, 44, 2636.1, 2636.0),
 (45, 88, 2636.0, 2635.9),
 (89, 96, 2635.9, 2635.8),
 (97, 97, 2635.8, 2635.7000000000003),
 (98, 114, 2635.7000000000003, 2635.6000000000004),
 (115, 115, 2635.5, 2635.6),
 (116, 134, 2635.7000000000003, 2635.6000000000004),
 (135, 168, 2635.5, 2635.6),
 (169, 170, 2635.6000000000004, 2635.7000000000003),
 (171, 179, 2635.7000000000003, 2635.8),
 (180, 203, 2635.8, 2635.9),
 (204, 207, 2635.9, 2636.0),
 (208, 215, 2636.0, 2636.1),
 (216, 226, 2636.1000000000004, 2636.2000000000003),
 (227, 237, 2636.2000000000003, 2636.3),
 (238, 239, 2636.4, 2636.3),
 (240, 240, 2635.8, 2635.9),
 (241, 248, 2636.3, 2636.4),
 (249, 255, 2636.4, 2636.5),
 (256, 263, 2636.5, 2636.6),
 (264, 266, 2636.6000000000004, 2636.7000000000003),
 (267, 274, 2636.7000000000003, 2636.8),
 (275, 41

In [8]:
fig.layout.shapes = [{
     'fillcolor': 'green' if c[3] > c[2] else 'red',
     'line': {'color': 'black', 'width': 1},
     'opacity': 0.2,
     'type': 'rect',
     'x0': c[0],
     'x1': c[1],
     'y0': c[2],
     'y1': c[3]
 } for c in C]